In [1288]:
import os
import warnings

import pandas as pd
import numpy as np


from jre_utils.datapath import (
    DATA_DIRECTORY_PATH,
)
from jre_utils.config import asset_types
from jp_prefecture.jp_cities import JpCity, jp_cities
from geopy.geocoders import Nominatim

from sklearn.preprocessing import StandardScaler

warnings.filterwarnings("ignore")
pd.set_option("display.max_columns", None)

In [1262]:
def area_to_int(area):
    if area == "2,000 m^2 or greater.":
        return 2000
    elif area == "5000 m^2 or greater.":
        return 5000
    else:
        return int(area)


def map_time_units(x):
    mapping = {
        "30-60minutes": 45,
        "1H-1H30": 75,
        "1H30-2H": 105,
        "2H-": 135,
    }
    
    return int(x) if x.isdigit() else mapping[x]

def map_layout(x):
    if x == "na":
        return "na"

    x = x.split("+")[0]

    if x[0].isdigit() and int(x[0]) == 1:
        if x != "1K" and x != "1LDK" and x != "1DK":
            return "1other"
        return x
    
    if x[0].isdigit() and int(x[0]) == 2:
        if x != "2LDK" and x != "2DK":
            return "2other"
        return x

    if x[0].isdigit() and int(x[0]) > 2:
        return f"{min(int(x[0]), 5)}LDK"
    
    return "other"

def map_land_shape(x):
    x = x.lower()
    x = x.replace("semi-", "")
    return x

def map_frontage(x):
    if x == "na":
        return 0
    if x == "50.0m or longer.":
        return 55
    return int(x.split(".")[0])

def map_floor_area(x):
    if "less" in x:
        return 10
    elif "greater" in x:
        return 2000
    else:
        return int(x)
    
def map_year_of_construction(x):
    if x == "before the war":
        return 1930
    
    return int(x)

def map_building_structure(x):
    if len(x.split(",")) > 1:
        return "combo"
    else:
        return x
    
def map_combined_use(x):
    if "Factory" in x:
        return "Factory"
    elif "Warehouse" in x:
        return "Warehouse"
    elif "Parking Lot" in x:
        return "Parking Lot"
    elif "Office" in x:
        return "Office"
    elif "Housing Complex" in x:
        return "Housing Complex"
    else:
        return x.split(",")[0]

In [1263]:
import json
from pprint import pprint

sub_city_to_city_path = f"{DATA_DIRECTORY_PATH}/core_scraped/sub_city_to_city.json"
with open(sub_city_to_city_path) as fd:
     sub_city_to_city = json.load(fd)
     pprint(f"E.g. Maps 1101 to {sub_city_to_city['1101']}")

area_code_to_area_path = f"{DATA_DIRECTORY_PATH}/core_scraped/area_code_to_area.json"
with open(area_code_to_area_path) as fd:
     area_code_to_area = json.load(fd)
     pprint(f"E.g. Maps 1100 to {area_code_to_area['1100']}") 

def get_city_code(area_code):
     return sub_city_to_city.get(area_code, area_code)

def get_area_from_area_code(area_code):
     return area_code_to_area.get(area_code, "na" )

def get_city_geocode(area_code):
    area_code = str(area_code)
    try:
        return tuple(jp_cities.citycode2geodetic(area_code)) 
    except:
        print(f"Could not find geocode for {area_code}")
        return np.NaN, np.NaN
    
def find_town_jp(all_towns_df, city_code, town_name, log=False):
    city_df = all_towns_df[all_towns_df["cityCode"] == int(city_code)]
    town_df = city_df[city_df["townAlphabet"].str.contains(town_name)]
    
    if town_df.empty:
        if log:
            print(f"JP could not find {town_name} in {city_code}")
        return None, None
    
    return town_df["longitude"].mean(), town_df["latitude"].mean()

def find_town_geopy(geolocator, address, log=False):
    location_info = geolocator.geocode(address)

    if not location_info:
        if log:
            print(f"Geopy could not find {address}")
        return None, None
    
    return location_info.longitude, location_info.latitude

def get_town_coordinates_jp(all_towns_df, city_name, city_code, town_name, log=False):
    geolocator = Nominatim(user_agent="my_app")
    
    # Try to locade in df - this is fast
    jp_lon, jp_lat = find_town_jp(all_towns_df, city_code, town_name, log)

    if not jp_lon or not jp_lat:
        address = f"{town_name}, {city_name}, Japan"
        
        # Fall back and try to locate with geopy - this is slow
        geopy_lon, geopy_lat = find_town_geopy(geolocator, address, log)
        
        if not geopy_lon or not geopy_lat:
            # if nothing works, just return the city coordinates
            return get_city_geocode(city_code)
        
        return geopy_lon, geopy_lat

    return jp_lon, jp_lat



'E.g. Maps 1101 to 1100'
'E.g. Maps 1100 to Hokkaido Sapporo-shi'


In [1264]:
prefecture_code = 13
trade_prices_data_path = f"{DATA_DIRECTORY_PATH}/core"

trade_prices_data_paths = [
    f"{trade_prices_data_path}/{filename}"
    for filename in sorted(os.listdir(trade_prices_data_path))
]
trade_prices_data_paths[prefecture_code - 1]

'../../data/core/13_Tokyo_20053_20233.csv'

In [1265]:
df = pd.read_csv(
    trade_prices_data_paths[prefecture_code - 1],
    encoding="unicode_escape",
    index_col="No",
)
df["area_code"] = df["City,Town,Ward,Village code"].astype(str)

# we may want to skip the following step in the future
df["area_code"] = df["area_code"].apply(get_city_code).astype(str)
df["area"] = df["area_code"].apply(get_area_from_area_code)

df["trade_price"] = df["Transaction-price(total)"]
df["trade_area"] = df["Area(m^2)"].apply(area_to_int)
df["unit_price"] = df["Transaction-price(Unit price m^2)"]
df["trade_price_per_area"] = df["trade_price"] / df["trade_area"]

df["quarter"] = df["Transaction period"].apply(lambda x: int(x.split(" ")[0][0]))
df["year"] = df["Transaction period"].apply(lambda x: int(x.split(" ")[2]))

df["unit_price"] = np.where(
    df["unit_price"].isna(),
    df["trade_price_per_area"],
    df["unit_price"],
)

df = df[
    df["Type"].isin(
        [
            asset_types["building"]["label"],
            asset_types["land"]["label"],
            asset_types["condo"]["label"],
        ],
    )
]

# Renaming

df = df.rename(columns = {
    "Type": "asset_type",
    "Region": "neighbourhood_classification",
    "Area": "subarea",
    "Nearest stationFName": "nearest_station",
    "Nearest stationFDistance(minute)": "time_to_nearest_station",
    "Layout": "layout",
    "Land shape": "land_shape",
    "Frontage": "frontage",
    "Total floor area(m^2)": "total_floor_area",
    "Year of construction": "year_of_construction",
    "Building structure": "building_structure",
    "Use": "use",
    "Purpose of Use": "purpose",
    "Frontage roadFDirection": "frontage_road_direction",
    "Frontage roadFClassification": "frontage_road_classification",
    "Frontage roadFBreadth(m)": "frontage_road_breadth",
    "City Planning": "zone",
    "Maximus Building Coverage Ratio(%)": "max_building_coverage_ratio",
    "Maximus Floor-area Ratio(%)": "max_floor_area_ratio",
    "Renovation": "renovation_status",
})

# Process factors
df["subarea"] = df["subarea"].fillna("")
df["neighbourhood_classification"] = df["neighbourhood_classification"].fillna("na")
df["nearest_station"] = df["nearest_station"].fillna("na")
df["time_to_nearest_station"] = df["time_to_nearest_station"].fillna("30-60minutes").apply(map_time_units)
df["layout"] = df["layout"].fillna("na").apply(map_layout)
df["land_shape"] = df["land_shape"].fillna("na").map(map_land_shape)
df["frontage"] = df["frontage"].fillna("na").apply(map_frontage)

df["total_floor_area"] = np.where(
    df["total_floor_area"].isna(),
    df["trade_area"].astype(str),
    df["total_floor_area"],
)

df["total_floor_area"] = df["total_floor_area"].apply(map_floor_area)


df["year_of_construction"] = np.where(
    df["year_of_construction"].isna(),
    (df["year"] - 30).astype(str),
    df["year_of_construction"],
)

df["year_of_construction"] = df["year_of_construction"].apply(map_year_of_construction)
df["age"] = df["year"] - df["year_of_construction"]
df["building_structure"] = df["building_structure"].fillna("na").map(map_building_structure)
df["frontage_road_direction"] = df["frontage_road_direction"].fillna("na")
df["frontage_road_classification"] = df["frontage_road_classification"].fillna("na")
df["frontage_road_breadth"] = df["frontage_road_breadth"].fillna("0.0").astype(float)
df["zone"] = df["zone"].fillna("na")
df["max_building_coverage_ratio"] = df["max_building_coverage_ratio"].fillna(0)
df["max_floor_area_ratio"] = df["max_floor_area_ratio"].fillna(0)
df["renovation_status"] = df["renovation_status"].fillna("na")

df["combined_use"] = np.where(
    df["purpose"].isna(),
    df["use"],
    df["purpose"],
)
df["combined_use"] = df["combined_use"].fillna("na").apply(map_combined_use)

df = df.drop(columns = [
    "City,Town,Ward,Village code",
    "City,Town,Ward,Village",
    "Transaction-price(total)",
    "Area(m^2)",
    "Transaction-price(Unit price m^2)",
    "trade_price_per_area",
    "Transaction period",
    "Prefecture",
    "Transactional factors",
    "year_of_construction", 
    "use",
    "purpose"
])


In [1266]:
# Advanced preprocessing

# Convert towns into Longitude and Latitude
jp_towns = JpCity(enable_town=True)
towns_df = jp_towns.towns
towns_df["townAlphabet"] = towns_df["townAlphabet"].fillna("")

towns_list_df = df[["area", "area_code", "subarea"]].drop_duplicates()
towns_list_df[["long", "lat"]] = towns_list_df.apply(
    lambda x: pd.Series(get_town_coordinates_jp(towns_df, x["area"], x["area_code"], x["subarea"])),
    axis=1,
)

df = df.merge(towns_list_df, on=["area", "area_code", "subarea"], how="left")


In [1267]:
# For each asset type, for each area_code
# 1. Run clustering algorithm and get cluster code for each transaction
# 2. Run regression to identify weights for each year

In [1430]:
# Land columns

numerical_columns = [
    "long",
    "lat",
    "time_to_nearest_station",
    "total_floor_area",
    "trade_area",
    "age",
    "frontage",
    "frontage_road_breadth",
    "max_building_coverage_ratio",
    "max_floor_area_ratio",
]

categorical_columns = [
    "neighbourhood_classification",
    "quarter",
    "zone",
    "renovation_status",
    "combined_use",
    "layout",
    "building_structure",
    "land_shape",
    # "frontage_road_direction",
    # "frontage_road_classification",
]

id_columns = ["year"]
metric_columns = ["unit_price_log"]

columns = numerical_columns + categorical_columns


In [1431]:
land_df = df[df["asset_type"] == asset_types["land"]["label"]]
building_df = df[df["asset_type"] == asset_types["building"]["label"]]
condo_df = df[df["asset_type"] == asset_types["condo"]["label"]]

In [1432]:
# area_df = land_df[land_df["area_code"] == "13101"].reset_index(drop=True)
area_df = building_df[building_df["area_code"] == "13101"].reset_index(drop=True)
area_df["year"] = area_df["year"].astype(str)
area_df["quarter"] = area_df["quarter"].astype(str)
area_df[f"unit_price_log"] = np.log(area_df["unit_price"] + 1)

In [1429]:
for column in numerical_columns:
    area_df[column] = np.log(area_df[column] + 1)

# Scale numerical variables
scaler = StandardScaler()
area_df[numerical_columns] = scaler.fit_transform(area_df[numerical_columns])

# Categorical variables
area_df = pd.get_dummies(area_df[columns + id_columns + metric_columns], columns=categorical_columns)
drop_columns = [col for col in area_df.columns if "na" in col]
area_df = area_df.drop(columns=drop_columns)
area_df

ValueError: Input X contains infinity or a value too large for dtype('float64').

In [1424]:
X_ord = area_df.drop(columns=id_columns + metric_columns)

In [1425]:
from sklearn.decomposition import PCA

# Perform PCA for dimensionality reduction
pca = PCA(n_components=0.90)  # Retain 95% of the variance
X_pca = pca.fit_transform(X_ord)

# Print the number of components retained by PCA
print("Number of components retained by PCA:", pca.n_components_)
print(X_pca.shape)

Number of components retained by PCA: 9
(315, 9)


In [1426]:
yearly_df = area_df[id_columns + metric_columns]
yearly_df = pd.get_dummies(yearly_df, columns=id_columns)
yearly_df = pd.concat(
    [
        yearly_df,
        pd.DataFrame(X_pca, columns=[f"PC{i+1}" for i in range(X_pca.shape[1])]),
    ],
    axis=1,
)
yearly_df

,unit_price_log,year_2005,year_2006,year_2007,year_2008,year_2009,year_2010,year_2011,year_2012,year_2013,year_2014,year_2015,year_2016,year_2017,year_2018,year_2019,year_2020,year_2021,year_2022,year_2023,PC1,PC2,PC3,PC4,PC5,PC6,PC7,PC8,PC9
0,15.687313,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0.296387,0.205170,-0.388837,1.506292,0.163370,3.002410,-0.147497,-0.227663,-0.962639
1,15.404746,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,-0.320348,-0.039934,-0.214520,0.802440,0.007693,1.098975,0.126730,-0.905790,-0.361875
2,15.607270,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0.867513,-0.030192,-0.374634,0.625158,-0.087014,-0.351827,0.647694,-0.724316,0.178729
3,14.845130,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,-1.927809,1.469196,0.955657,0.457797,0.461749,0.358570,1.135893,0.324113,0.231455
4,15.201805,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,-0.336708,-0.821013,-0.607515,1.519816,1.444096,-0.445919,0.157887,0.629655,-0.305895
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
310,14.690980,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-2.153528,1.555820,1.196973,-0.813743,-0.556498,-0.372107,1.277803,0.113822,0.533719
311,13.795309,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.633113,1.710122,0.489382,0.223671,-0.804513,1.592425,0.091620,-0.714876,-0.431253
312,14.508658,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-1.135261,0.140575,0.147155,-1.274966,-0.068390,-0.091653,-0.582197,-0.281464,-0.838588
313,11.918397,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1.190307,1.926048,0.827396,-1.938729,-3.203943,-0.703787,-0.562088,0.471184,0.850269


In [1427]:
import statsmodels.api as sm

X = yearly_df.drop(columns=metric_columns)
y = yearly_df[metric_columns[0]]

# Add a constant to the model (the intercept)
X = sm.add_constant(X)

# Fit the OLS model
model = sm.OLS(y, X).fit()
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:         unit_price_log   R-squared:                       0.594
Model:                            OLS   Adj. R-squared:                  0.556
Method:                 Least Squares   F-statistic:                     15.54
Date:                Thu, 28 Mar 2024   Prob (F-statistic):           1.05e-41
Time:                        16:12:44   Log-Likelihood:                -207.12
No. Observations:                 315   AIC:                             470.2
Df Residuals:                     287   BIC:                             575.3
Df Model:                          27                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         13.7502      0.030    459.123      0.0

In [1368]:
from sklearn.linear_model import LinearRegression

# Create and fit the linear regression model
model = LinearRegression()
model.fit(X, y)

year_columns = [col for col in X.columns if "year" in col]
# Get the weights (coefficients) for each year and PCA component
year_weights = model.coef_[:len(year_columns)]
pca_weights = model.coef_[len(year_columns):]

# Create a DataFrame to store the year weights (price index)
# year_weights_df = pd.DataFrame({'Year': year_encoder.categories_[0][1:], 'Weight': year_weights})
# year_weights_df.set_index('Year', inplace=True)

# # Print the year weights (price index)
# print("Year Weights (Price Index):")
# print(year_weights_df)

# # Print the PCA component weights
# print("\nPCA Component Weights:")
# for i, weight in enumerate(pca_weights, start=1):
#     print(f"PC{i}: {weight:.4f}")

In [1369]:
year_weights

array([-2.87822706e+08,  5.24127037e+11,  5.24127037e+11,  5.24127037e+11,
        5.24127037e+11,  5.24127037e+11,  5.24127037e+11,  5.24127037e+11,
        5.24127037e+11,  5.24127037e+11,  5.24127037e+11,  5.24127037e+11,
        5.24127037e+11,  5.24127037e+11,  5.24127037e+11,  5.24127037e+11,
        5.24127037e+11,  5.24127037e+11,  5.24127037e+11])

In [1271]:
from jre_utils.visualize import plot_time_series


# plot_time_series(combined_cluster_df, column="annualized_return", group_by_columns=["cluster_code", "year"], granularity_columns=["cluster_code"], title="Returns by cluster")

In [ ]:
# Now that I have a dense vector - I can append my year dummies and compute the returns for each year + pca